# Calculate compression for WorkingMemory module
This notebook is used to calculate the compression ratio and 10 other statistics for a preprocessed SPAUN WorkingMemory module. The module is one of several used in the SPAUN model and is used in this notebook to illustrate the compression we can achieve with SPAUN. 

When calculating the statistics for a single preprocessed SPAUN module, bear in mind, the modules inputs and outputs from other modules may not be instantiated and, therefore, the Nodes and Ensembles in WorkingMemory may not be connected to Nodes and Ensembles from other modules. If this is the case, preprocessing can eliminate Nodes if the Nodes have no input connection as well as encoding or decoding Ensemble weights if the Ensemble has no input or output connection respectively. If a Node or encoding/decoding weight is eliminated, the statistics (e.g. the total encoding weights) are smaller than they would be when the WorkingMemory module is instantiated in a full SPAUN module. 

The following sections
- provide formulas for the compression ratio and the 10 other statistics we are interested, 
- provide statistics for a preprocessed SPAUN WorkingMemory module with 512-dimensional semantic pointers.
- verify the results

## Statistics
This section provides formulas for the the statistics we are interested in. The formulas use the following four variables. 

$D$ = semantic pointer dimensions

$k_1$  = number of neurons per Ensemble dimension

$k_2$ = number of neurons per Ensemble dimension

$d_1$ = input/output dimensions of an Ensemble 

The variable $D$ is provided to Nengo when instantiating SPAUNs WorkingMemory module. The variables $k_1$, $k_2$ and $d_1$ are not provided by the user during instantiation but are contained in the Nengo algorithms which instantiate and preprocess the WorkingMemory module.

The 11 statistics we are interested in are as follows.

### Total decoding weights
The total number of decoding weights is calculated as follows:<br><br>

$$
N_\text{D} = (73 + \frac{59D}{d_1})k_1d_1^2 + 3k_2d_1^2
$$


### Total encoding weights
The total number of encoding weights is calculated as follows:<br><br>

$$
N_\text{E} = (63 + \frac{95D}{d_1})k_1d_1^2 + 3k_2d_1^2
$$

### Total transform resource count
The total transform resource count is calculated as follows:<br><br>

$$
N_\text{T} =
\begin{cases}
(212D+9) + 3D^2 + 24 & \text{if }  D==2 \text{ or } D==3 \text{ or } D==4 \\
(212D+9) + 6(D-3)(D-4) + 3D^2 + 24 & \text{if } D < 2 \text{ or } D > 4
\end{cases}
$$

Note that the last two terms are due to Bias nodes that in the module that grow with dimensions D and are not part of a connection path between Ensembles. Therefore, we will not see these terms in the equations for synaptics weight or fanout.

### Total number of synapses
The total number of synapses is calculated as follows:<br><br>

$$
N_\text{SYN}=
\begin{cases}
(206D+9)(k_1d_1)^2 + 6Dk_1k_2d_1^2 & \text{if }D==2, D==3, D==4 \\
(206D+9)(k_1d_1)^2 + 6Dk_1k_2d_1^2 + 6(D-3)(D-4)(k_1d_1)^2 & \text{if }D < 2 \text{ or } D > 4
\end{cases}
$$

### Compression ratio
The compression ratio is calculated as follows:<br><br>

$$C=\frac{N_\text{E}+N_\text{D}+N_\text{T}}{N_\text{SYN}}$$

### Total number of neurons
The total number of neurons is calculated as follows:<br><br>

$$
N_\text{NRN} = (73 + \frac{52D}{d_1})k_1d_1 + 3k_2d_1
$$

### Total number of ensembles
The total number of ensembles is calculated as follows:<br><br>

$$
N_\text{ENS} = 76 + \frac{58D}{d_1}
$$

### Fanout

The total fanout is calculated as follows:<br><br>

$$
N_\text{SYN}=
\begin{cases}
(212D+9) & \text{if }D==2, D==3, D==4 \\
(212D+9) + 6(D-3)(D-4) & \text{if }D < 2 \text{ or } D > 4
\end{cases}
$$

### Neurons per ensemble
The ratio of neurons to ensembles is defined as:<br><br>

$$R_\text{NRN-ENS}=\frac{N_\text{NRN}}{N_\text{ENS}}$$

###Synapses per neurons
The ratio of synapses to neurons is defined as:<br><br>

$$R_\text{SYN-NRN}=\frac{N_\text{SYN}}{N_\text{NRN}}$$

### Average fanout

$$R_\text{FAN-ENS}=\frac{N_\text{FAN}}{N_\text{ENS}}$$

## Working memory module with 512-D semantic pointers

We can now evaluate the compression ratio for the WorkingMemory module in SPAUN. Given that the number of dimensions D is 512, input/output dimensions of every Ensemble $d_1$ is 1 and the constant $k_2$ is equal to $\frac{2}{5}k_1$, we can use those values to simplify the equations $N_\text{D}$, $N_\text{E}$, $N_\text{T}$ and $N_\text{SYN}$ as follows.

For the equation $N_\text{D}$, we replace $k_2$ and $d_1$ and re-factor, getting 

$$
N_\text{D} = (74.2 + 59D)k_1
$$

For the equation $N_\text{E}$, we again replace $k_2$ and $d_1$ and re-factor, getting 

$$
N_\text{E} = (64.2 + 95D)k_1
$$

For the equation $N_\text{T}$, since $D == 512$, we choose the second case, getting 

$$
N_\text{T} = (212D + 9) + 6(D-3)(D-4) + 3D^2 + 24
$$

For reasons that will become clear shortly, let's expand $N_\text{T}$ getting

$$
N_\text{T} = 9D^2 + 170D + 105
$$

For equation $N_\text{SYN}$, since $D == 512$, we choose the second case, replace $k_2$ and $d_1$, and refactor, getting

$$
N_\text{SYN} = ((208.4D+9) + 6(D-3)(D-4))k_1^2
$$

We'll expand $N_\text{SYN}$ as we did for $N_\text{T}$ getting

$$
N_\text{SYN} = (6D^2 + 166.4D + 81)k_1^2
$$

Finally, the equation $C$ is composed of the first three equations in the numerator and the last equation in the denominator. I'll write $C$ as a summation of the three terms getting

$$
C = \frac{(74.2 + 59D)k_1}{(6D^2 + 166.4D + 81)k_1^2 } + \frac{(64.2 + 95D)k_1}{(6D^2 + 166.4D + 81)k_1^2} + \frac{9D^2 + 170D + 105}{(6D^2 + 166.4D + 81)k_1^2}
$$

Simplifying

$$
C = \frac{(74.2 + 59D)}{(6D^2 + 166.4D + 81)}\frac{1}{k_1} + \frac{(64.2 + 95D)}{(6D^2 + 166.4D + 81)}\frac{1}{k_1} + \frac{9D^2 + 170D + 105}{(6D^2 + 166.4D + 81)}\frac{1}{k_1^2}
$$

We notice two things about compression with this module. First, the synaptics weight count grows quadratically with respect to semantic pointer dimensions. Second, the encoder and decoder weights in the first two terms grow linearly whereas the transform resource count in the third term grows quadratically with respect to semantic pointer dimensions. Third, the first two terms can dominate compression compared to the third term when when $D > k_1$. 

In SPAUN, $D$ is 512 and $k_1=50$, we would expect compression to be dominated by a compression of encoding and decoding weights opposed to compression in transform resource counts.

Another way to look at this is to take $\frac{1}{C}$, the compression ratio of synaptic weights over encoding weights, decoding weights and transform resource counts. In this case, the formula for compression is 

$$
\frac{1}{C} = \frac{(6D^2 + 166.4D + 81)}{(74.2 + 59D)}k_1 + \frac{(6D^2 + 166.4D + 81)}{(64.2 + 95D)}k_1 + \frac{(6D^2 + 166.4D + 81)}{9D^2 + 170D + 105}k_1^2
$$

We can simplify this knowing the rate of growth of the first two terms is linear and the third term is constant with respect to $D$, getting

$$
\frac{1}{C} = \mathcal{O}(D)k_1 + \mathcal{O}(D)k_1 + \mathcal{O}(1)k_1^2
$$

Since $k_1$ is a constant, we would expect the first two terms growth rate to exceed the third term, therefore, we would expect the compression ratio of synaptic weights to encoding weights and decoding weights to exceed the compression ratio of synaptic weights to tranform resource counts.

## Implementation
The following implements the formula's above on a SPAUN model with only the WorkingMemory module instantiated. The parameter D can be set for testing.

In [1]:
%%capture
D=10
%run build_spaun_wm_only.py -d {D}
import nengo_brainstorm_pp as pp
import nengo

pp_model = pp.preprocess(model, find_io = False)

In [2]:
from nengo_brainstorm_pp import gv_utils
#gv_utils.gv_plot(pp_model,size=(50,100))

In [3]:
import numpy as np

k_1 = 50
d_misc = 1
k_2 = 20
d_mem = 1
d_sel = 1
d_gate = 1
#D = 4
d_1 = 1
d_ne=1

###################
# Calculating number of encoders
###################
def _wm_misc_blocks_num_encoders(k_1, d_misc):
    return 0

def _wm_mem_blocks_num_encoders(D, k_1, d_1, d_mem):
    return (7 * ((2*k_1*(d_1**2)) + 2*((3*k_1*(d_1**2))+3*((D/d_1)*k_1*(d_mem**2)) \
                                       + 2*((D/d_1)*k_1*(d_ne**2)))))

def _wm_sel_blocks_num_encoders(D,k_1, d_1, k_2, d_sel):
    return (3 * (k_2*(d_1**2) + 2*((D/d_1)*k_1*(d_sel**2)) \
                 + (4*(D/d_1)*k_1*(d_ne**2))))

def _wm_gate_blocks_num_encoders(D,k_1,d_1,d_gate):
    return (7 * (D/d_1)*k_1*(d_gate**2)) + k_1*(d_gate**2)

def wm_num_encoders(D,k_1,d_1,k_2,d_misc,d_mem,d_sel,d_gate):
    return _wm_misc_blocks_num_encoders(k_1, d_misc) + \
        _wm_mem_blocks_num_encoders(D, k_1, d_1, d_mem) + \
        _wm_sel_blocks_num_encoders(D,k_1, d_1, k_2, d_sel) + \
        _wm_gate_blocks_num_encoders(D,k_1,d_1,d_gate)   
        

###################
# Calculating number of decoders
###################
def _wm_misc_blocks_num_decoders(k_1, d_1):
    return 3*k_1*(d_1**2)

def _wm_mem_blocks_num_decoders(D, k_1, d_1, d_mem):
    return ((63+42*(D/d_1))*k_1*d_1**2) + (D/d_1)*k_1*(d_1**2)

def _wm_sel_blocks_num_decoders(D,k_1, d_1, k_2, d_sel):
    return (3 * ((1*k_2*(d_1**2)) + 3*((D/d_1)*k_1*(d_sel**2))))

def _wm_gate_blocks_num_decoders(D,k_1,d_1,d_gate):
    return (7 * ((1*k_1*(d_1**2)) + ((D/d_1)*k_1*(d_gate**2))))

def wm_num_decoders(D,k_1,d_1,k_2,d_misc,d_mem,d_sel,d_gate):
    return _wm_misc_blocks_num_decoders(k_1, d_1) + \
        _wm_mem_blocks_num_decoders(D, k_1, d_1, d_mem) + \
        _wm_sel_blocks_num_decoders(D,k_1, d_1, k_2, d_sel) + \
        _wm_gate_blocks_num_decoders(D,k_1,d_1,d_gate)        

###################
# Calculating transform resource counts
###################

def wm_transform_count(D):
    result = (212*D+9) + 3*(D**2) + 24
    if (D<2 or D>4):
        result += (6*(D-3)*(D-4))
    return result
###################
# Calculating synaptic weights
###################
# Linear regression plus knowing that some subnets are 50*50 and some are 50*20; expand on this later

def wm_synaptic_weights(D,k_1,d_1,k_2):
    result = (206*D+9)*((k_1*d_1)**2) + 6*D*(k_1*d_1)*(k_2*d_1)
    if (D<2 or D>4):
        result += (6*(D-3)*(D-4))*((k_1*d_1)**2)
    return result

###################
# Calculating compression ratio
###################

def wm_compression(D,k_1,d_1,k_2,d_misc,d_mem,d_sel,d_gate):
    return (wm_num_decoders(D,k_1,d_1,k_2,d_misc,d_mem,d_sel,d_gate) \
            + wm_num_encoders(D,k_1,d_1,k_2,d_misc,d_mem,d_sel,d_gate) \
            + wm_transform_count(D)) / \
            float(wm_synaptic_weights(D,k_1,d_1,k_2))
           
###################
# Calculating number of neurons
###################
def _wm_misc_blocks_num_neurons(k_1, d_misc):
    return 3*k_1*d_misc

def _wm_mem_blocks_num_neurons(D, k_1, d_1, d_mem):
    return (7 * ((3*k_1*d_1) + 2*((3*k_1*d_1)+3*((D/d_1)*k_1*d_mem))))

def _wm_sel_blocks_num_neurons(D,k_1, d_1, k_2, d_sel):
    return (3 * ((1*k_2*d_1) + 3*((D/d_1)*k_1*d_sel)))

def _wm_gate_blocks_num_neurons(D,k_1,d_1,d_gate):
    return (7 * ((1*k_1*d_1) + ((D/d_1)*k_1*d_gate)))

def wm_num_neurons(D,k_1,d_1,k_2,d_misc,d_mem,d_sel,d_gate):
    return _wm_misc_blocks_num_neurons(k_1, d_misc) + \
        _wm_mem_blocks_num_neurons(D, k_1, d_1, d_mem) + \
        _wm_sel_blocks_num_neurons(D,k_1, d_1, k_2, d_sel) + \
        _wm_gate_blocks_num_neurons(D,k_1,d_1,d_gate)        
        
###################
# Calculating number of ensembles
###################
def _wm_misc_blocks_num_ensembles():
    return 3

def _wm_mem_blocks_num_ensembles(D, d_1):
    return 63 + 42*(D/d_1)

def _wm_sel_blocks_num_ensembles(D, d_1):
    return 3 + 9*(D/d_1)
#    return (3 * (1 + 3*(D/d_1)))

def _wm_gate_blocks_num_ensembles(D, d_1):
    return 7 + 7*(D/d_1)

def wm_num_ensembles(D,d_1):
    return _wm_misc_blocks_num_ensembles() + \
        _wm_mem_blocks_num_ensembles(D,d_1) + \
        _wm_sel_blocks_num_ensembles(D,d_1) + \
        _wm_gate_blocks_num_ensembles(D,d_1)

###################
# Calculating fanout
###################
def wm_fanout(D,k_1,d_1,k_2):        
    result = (212*D+9)
    if (D<2 or D>4):
        result += (6*(D-3)*(D-4))
    return result

N_NRN = wm_num_neurons(D,k_1,d_1,k_2,d_misc,d_mem,d_sel,d_gate)
N_ENS = wm_num_ensembles(D,d_1)
N_D = wm_num_decoders(D,k_1,d_1,k_2,d_misc,d_mem,d_sel,d_gate)
N_E = wm_num_encoders(D,k_1,d_1,k_2,d_misc,d_mem,d_sel,d_gate)
N_T = wm_transform_count(D)
N_SYN = wm_synaptic_weights(D,k_1,d_1,k_2)
C = wm_compression(D,k_1,d_1,k_2,d_misc,d_mem,d_sel,d_gate)
N_FAN = wm_fanout(D,k_1,d_1,k_2)
R_NRN_ENS = float(N_NRN/float(N_ENS))
R_SYN_NRN = float(N_SYN/float(N_NRN))
R_FAN_ENS = float(N_FAN/float(N_ENS))
print "WM number of neurons = ",N_NRN
print "WM number of ensembles = ", N_ENS
print "WM decoding weights =", N_D
print "WM encoding weights =", N_E
print "WM transform resource count=", N_T
print "WM number of synapses = ", N_SYN
print "WM compression ratio = %.8f" % C
print "WM fanout = ", N_FAN
print "Avg. neurons per ensemble = %.2f" % R_NRN_ENS
print "Avg. synapses per neuron = %.2f " % R_SYN_NRN
print "Avg. fanout = %.2f" % R_FAN_ENS

WM number of neurons =  32710
WM number of ensembles =  656
WM decoding weights = 33210
WM encoding weights = 50410
WM transform resource count= 2705
WM number of synapses =  5862500
WM compression ratio = 0.01472495
WM fanout =  2381
Avg. neurons per ensemble = 49.86
Avg. synapses per neuron = 179.23 
Avg. fanout = 3.63


## Verification
We now want to verify that the results from our formulas are correct. To verify correctness, we use the compute_stats function which calculates the values we are interested in by counting the number of relevant elements for each statistic. For example, to determine the number of ensembles, the compute_stats function simply counts the number of ensembles in the model. In another example, to determine the total number of synapses in the model, the function evaluates each ensemble determining the list of ensembles connected to the given ensemble (the fanout). After determining the fanout, the function computes the synaptic weights between the ensemble and each ensemble it is connected to. The function repeats this for each ensemble.

The compute_stats function returns a dictionary of values corresponding to the values we computed using this notebooks formulas. The formulas results are compared to the dictionary values in the compare_stats function. The compare_stats function will tell if the formula results and dictionary values are equal (or approximately equal in the case of ratios) which would indicate that the formulas are correct.

In [4]:
import compute_stats as cp
stats = cp.compute_stats(pp_model,print_results=False)
def compare_stats(N_NRN, N_ENS, N_D, N_E, N_T, N_SYN, C, N_FAN, R_NRN_ENS, R_SYN_NRN, R_FAN_ENS, stats):
    all_passed = True
    if (abs(N_NRN - stats['N_NRN']) > 0):
        print "N_NRN test fails"
        all_passed = False      
    if (abs(N_ENS - stats['N_ENS']) > 0):
        print "N_ENS test fails"
        all_passed = False    
    if (abs(N_D - stats['N_D']) > 0):
        print "N_D test fails"
        all_passed = False    
    if (abs(N_E - stats['N_E']) > 0):
        print "N_E test fails"
        all_passed = False    
    if (abs(N_T - stats['N_T']) > 0):
        print "N_T test fails"
        all_passed = False    
    if (abs(N_SYN - stats['N_SYN']) > 0):
        print "N_SYN test fails"
        all_passed = False    
    if (abs(C - stats['C']) > 0.00001):
        print "C test fails"
        all_passed = False    
    if (abs(N_FAN - stats['N_FAN']) > 0):
        print "N_FAN test fails"
        all_passed = False    
    if (abs(R_NRN_ENS - stats['R_NRN_ENS']) > 0.00001):
        print "R_NRN_ENS test fails"
        all_passed = False    
    if (abs(R_SYN_NRN - stats['R_SYN_NRN']) > 0.00001):
        print "R_SYN_NRN test fails"
        all_passed = False    
    if (abs(R_FAN_ENS - stats['R_FAN_ENS']) > 0.00001):
        print "R_FAN_ENS test fails"
        all_passed = False    
        
    if all_passed:
        print "All tests passed"
    else:
        print "Some tests failed"
        
compare_stats(N_NRN, N_ENS, N_D, N_E, N_T, N_SYN, C, N_FAN, R_NRN_ENS, R_SYN_NRN, R_FAN_ENS, stats)

All tests passed


As expected, the tests passed.

# Appendix
## Derivations for WorkingMemory module equations
### Total decoding weights

$$
N_\text{D} = 3k_1d_1^2 + (63 + \frac{42D}{d_1}) \cdot k_1d_1^2 + \frac{D}{d_1}k_1d_1^2 + 3k_2d_1^2 + \frac{9D}{d_1}k_1d_1^2 + 7k_1d_1^2 + \frac{7D}{d_1}k_1d_1^2
$$

The first term is the number of encoding weights from the two miscellaneous blocks. THe second and third terms are from the memory blocks. The fourth and fifth terms are from selector blocks and the last two terms are from the gate blocks.

### Total encoding weights
    
$$
N_\text{E} = 14k_1d_1^2 + 42k_1d_1^2 + \frac{42D}{d_1}k_1d_1^2 + \frac{28D}{d_1}k_1d_1^2 + 3k_2d_1^2 + \frac{6D}{d_1}k_1d_1^2 + \frac{12D}{d_1}k_1d_1^2 + \frac{7D}{d_1}k_1d_1^2 + k_1d_1^2
$$

The first term is the number of encoding weights from the two miscellaneous blocks. THe second and third terms are from the memory blocks. The fourth and fifth terms are from selector blocks and the last two terms are from the gate blocks.

### Total number of neurons
    
$$
N_\text{NRN} = 
3k_1d_1 + 63k_1d_1 + \frac{42D}{d_1}k_1d_1 + 3k_2d_1 + \frac{3D}{d_1}k_1d_1 + 7k_1d_1 + \frac{7D}{d_1}k_1d_1
$$

The first term is the number of neurons from the two miscellaneous blocks. THe second and third terms are from the memory blocks. The fourth and fifth terms are from selector blocks and the last two terms are from the gate blocks.

### Total number of ensembles

$$
N_\text{ENS} = 3 + 63 + \frac{42D}{d_1} + 3 + \frac{9D}{d_1} + 7 + \frac{7D}{d_1}
$$

The first term is the number of ensembles from the two miscellaneous blocks. THe second and third terms are from the memory blocks. The fourth and fifth terms are from selector blocks and the last two terms are from the gate blocks.

## Repo's used
The following repo versions were used when calculating these numbers:<br>

github.com:Stanford-BIS/spaun2.0.git<br>
SHA ID: 717c92f3e52641dc4e5e03ddc4d17e9934f8982f


github.com:ctn-waterloo/nef-chip-hardware.git<br>
SHA ID: a0a7cd470b5f0ca12c68de35ec0b28f437526bba

github.com:nengo/nengo.git<br>
SHA ID: a4e6cb6196e99b14d4165fab692d6b8eaab610cf
